<a href="https://colab.research.google.com/github/Soundarya0024/-A-Django-based-Construction-Record-Management-System/blob/main/Telecom_DSS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd

df_customers = pd.read_csv("/content/telecom_customer_churn.csv")
df_dict = pd.read_csv("/content/telecom_data_dictionary.csv", encoding='latin1')
df_population = pd.read_csv("/content/telecom_zipcode_population.csv")

df_customers.head()

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [34]:
#Data Cleaning (Business Focus)
# Fix negative monthly charges
df_customers['Monthly Charge'] = df_customers['Monthly Charge'].abs()

# Convert numeric columns
num_cols = ['Monthly Charge', 'Total Charges', 'Total Revenue']
for col in num_cols:
    df_customers[col] = pd.to_numeric(df_customers[col], errors='coerce')

# Handle missing churn reasons
df_customers['Churn Reason'].fillna("Not Churned", inplace=True)


/tmp/ipython-input-1659899652.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_customers['Churn Reason'].fillna("Not Churned", inplace=True)


In [35]:
#Create Business KPIs
total_customers = df_customers.shape[0]
churned = df_customers[df_customers['Customer Status'] == 'Churned'].shape[0]
churn_rate = (churned / total_customers) * 100

total_revenue = df_customers['Total Revenue'].sum()
avg_revenue = df_customers['Total Revenue'].mean()

print("Total Customers:", total_customers)
print("Churn Rate:", round(churn_rate,2), "%")
print("Total Revenue:", round(total_revenue,2))
print("Average Revenue per Customer:", round(avg_revenue,2))


Total Customers: 7043
Churn Rate: 26.54 %
Total Revenue: 21371131.69
Average Revenue per Customer: 3034.38


In [36]:
#Merge Population Data (UNIQUE PART)
df_final = df_customers.merge(
    df_population,
    how='left',
    left_on='Zip Code',
    right_on='Zip Code'
)




In [37]:
#Check Missing Values
df_customers.isnull().sum()


,0
Customer ID,0
Gender,0
Age,0
Married,0
Number of Dependents,0
City,0
Zip Code,0
Latitude,0
Longitude,0
Number of Referrals,0


In [38]:
#Fix negative monthly charges (business logic)
df_customers = df_customers[df_customers['Monthly Charge'] >= 0]


In [39]:
#Handle missing churn info
df_customers['Churn Category'] = df_customers['Churn Category'].fillna('Not Churned')
df_customers['Churn Reason'] = df_customers['Churn Reason'].fillna('Not Churned')


In [40]:
#Create Business KPIs
#ARPU (Average Revenue per User)
df_customers['ARPU'] = df_customers['Total Revenue'] / df_customers['Tenure in Months']
#Churn Flag (0/1)
df_customers['Churn_Flag'] = df_customers['Customer Status'].apply(
    lambda x: 1 if x == 'Churned' else 0
)
#Tenure Segments
df_customers['Tenure Segment'] = pd.cut(
    df_customers['Tenure in Months'],
    bins=[0, 12, 24, 48, 72, 100],
    labels=['0-1 yr','1-2 yr','2-4 yr','4-6 yr','6+ yr']
)

In [41]:
df_customers.to_csv("cleaned_telecom_churn.csv", index=False)


In [42]:
from google.colab import files
files.download("cleaned_telecom_churn.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
#Confirm Data Loaded Correctly
df_customers.shape



(7043, 41)

In [44]:
#Data Cleaning
#Fix negative & invalid values (you already saw -4.0 charge)
# Remove invalid monthly charges
df_customers = df_customers[df_customers['Monthly Charge'] >= 0]

# Convert Total Charges to numeric
df_customers['Total Charges'] = pd.to_numeric(
    df_customers['Total Charges'], errors='coerce'
)

df_customers['Total Charges'].fillna(
    df_customers['Total Charges'].median(), inplace=True
)


/tmp/ipython-input-2482440240.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_customers['Total Charges'].fillna(


In [45]:
#Feature Engineering (THIS MAKES IT UNIQUE)
# Churn flag
df_customers['Churn_Flag'] = df_customers['Customer Status'].apply(
    lambda x: 1 if x == 'Churned' else 0
)

# ARPU
df_customers['ARPU'] = df_customers['Total Revenue'] / (
    df_customers['Tenure in Months'] + 1
)

# Tenure segments
df_customers['Tenure_Segment'] = pd.cut(
    df_customers['Tenure in Months'],
    bins=[0, 12, 24, 48, 72, 100],
    labels=['0-1yr', '1-2yr', '2-4yr', '4-6yr', '6+yr']
)

In [46]:
#Business KPIs (INTERVIEW GOLD)
# Overall churn rate
churn_rate = df_customers['Churn_Flag'].mean() * 100
churn_rate


np.float64(26.536987079369588)

In [47]:
# Revenue by customer status
df_customers.groupby('Customer Status')['Total Revenue'].sum()


,Total Revenue
Customer Status,
Churned,3684459.82
Joined,54279.75
Stayed,17632392.12


In [48]:
# Churn by contract
df_customers.groupby('Contract')['Churn_Flag'].mean() * 100


,Churn_Flag
Contract,
Month-to-Month,45.844875
One Year,10.709677
Two Year,2.549124


In [49]:
#Export CLEAN DATA FOR POWER BI
df_customers.to_csv(
    "Telecom_Business_Decision_Data.csv",
    index=False
)
